In [ ]:
import pandas as pd
import gensim

In [ ]:
IN_DATA_DIR = "../../data/raw/"
INPUT_FILE_NAME = 'cleaned.parquet'
OUT_DATA_DIR = "../../data/processed/"

In [ ]:
df = pd.read_parquet(IN_DATA_DIR + INPUT_FILE_NAME)
df.head()
len(df)

In [ ]:
def count_tags(tag_column):
    tags = tag_column.str.replace(', ', ',').str.lower().str.strip()
    joined_tags = tags.str.cat(sep=',').split(',')
    all_tags_w_dup = pd.Series(joined_tags)

    tag_counts = all_tags_w_dup.value_counts()
    tag_list = list(tag_counts.index)
    return tag_counts, tag_list

In [ ]:
# split_tags = tags.str.split(',')
# tag_counts_per_talk = split_tags.apply(len)

tag_counts, tag_list = count_tags(df['tags'])

In [ ]:
tag_counts

In [ ]:
tag_cutoff = int(0.02*len(df.index))

squashed_tags = pd.DataFrame(tag_counts)
squashed_tags = squashed_tags[(squashed_tags[0]>tag_cutoff)]
squash_list = list(squashed_tags.index.values)


In [ ]:
squashed_tags

In [ ]:
#remove ted tags
ted_tags=[]
for tag in squash_list:
    if 'ted' in tag:
        ted_tags.append(tag)
print(ted_tags)
squashed_tags = squashed_tags.drop(labels = ted_tags)
squash_list = list(squashed_tags.index.values)

In [ ]:
squashed_tags
print(squash_list)

In [ ]:
def squashing (x,squash_list):
    original_tags = x
#     print(x)
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    final_tags = []
    for tag in split_tags:
        if tag in squash_list:
            
            final_tags.append(tag)
    final_string = ','.join(final_tags)
    return final_string

In [ ]:
df1 = df.copy()
df1['squash_tags'] = df1['tags']
df1['squash_tags'] = df1['squash_tags'].map(lambda x: squashing(x,squash_list))
df1 = df1[df1['squash_tags']!='']

In [ ]:
# df1[df1['squash_tags']==float('nan')]
df1[df1['squash_tags']==None]

In [ ]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [ ]:
nlp_list = []
for i in range(len(squash_list)):
    nlp_list.append(nlp(squash_list[i]))
# print(nlp_list)

In [ ]:
simlist = []
tagsimcount = [0]*len(squash_list)
tagsimdict = {el:[] for el in squash_list}
for outer in range(len(nlp_list)):
    for inner in range(len(nlp_list)):
        if inner<outer:
            sim = nlp_list[outer].similarity(nlp_list[inner])
            if sim > 0.65:
                ow = squash_list[outer]
                iw = squash_list[inner]
                ow_count = tag_counts.at[ow]
                iw_count = tag_counts.at[iw]
                simlist.append([ow,iw,sim])
                tagsimcount[outer] += 1
                tagsimcount[inner] += 1
                tagsimdict[ow].append(iw)
                tagsimdict[iw].append(ow)
                
items =list(tagsimdict.items())

for k,v in items:
    if v == []:
        del tagsimdict[k]             

# print(simlist)
# print(tagsimcount)
# print(tagsimdict)
sim_tags = list(tagsimdict.keys())

In [ ]:
# replace tags
# GENERALISE: 'education','teaching' -> 'education'
# REMAIN: 'africa', 'asia' -> 'africa', 'asia'

#anything with ted remove hahaha
print(tagsimcount)
print(tagsimdict)

In [ ]:
def sim_counts(a):
#     print(a)
    a_count=tag_counts.at[a]
    all_tags = [a]
    all_count = [a_count]
    b = tagsimdict[a]
    for word in b:
        b_count=tag_counts.at[word]
        all_tags.append(word)
        all_count.append(b_count)
    max_count = max(all_count)
    max_index = all_count.index(max_count)
    major = all_tags[max_index]
    minor = all_tags
    minor.remove(major)
    final = [major]+minor
    return final

In [ ]:
sim_tags_ordered = []
for tag in squash_list:
    if tag in sim_tags:
        sim_tags_ordered.append(tag)
# print(sim_tags_ordered)
sim_tags_reversed = sim_tags_ordered
sim_tags_reversed.reverse()
print(sim_tags_reversed)
print(len(sim_tags),len(sim_tags_reversed))

In [ ]:
final_replacement = {}
for i in range(len(sim_tags_reversed)):
    tag = sim_tags_reversed[i]
    sims = tagsimdict[tag]
    #check if words that main tag is similar too have similar words too
    for sim in sims:
        if sim in final_replacement.keys():
            simlist = final_replacement[sim]
            sims+=simlist
            del final_replacement[sim]
    #remove duplicates
    sims = list(dict.fromkeys(sims))
    final_replacement[tag]=sims
print(final_replacement)

In [ ]:
print(final_replacement.keys())

In [ ]:
def replacing (x):
    original_tags = x
    tags = original_tags.replace(', ', ',').lower().strip()
    split_tags = tags.split(',')
    final_tags = []
    for tag in split_tags:
        if tag in final_replacement.keys():
            #tag is a major
            final_tags.append(tag)
        else:
            #tag is a minor
            r = []
            for k,v in final_replacement.items():
                if tag in v:
                    #tag is a minor w/ a major
                    r.append(k)
            if r == []:
                r.append(tag)
            final_tags += r
    final_tags = list(dict.fromkeys(final_tags))
    final_string = ','.join(final_tags)
    return final_string

In [ ]:
df2 = df1.copy()
df2['squash2_tags'] = df2['squash_tags']
df2['squash2_tags'] = df2['squash2_tags'].map(lambda x: replacing(x))
df2 = df2[df2['squash2_tags']!='']

In [ ]:
df2[df2['squash2_tags']==float('nan')]

In [ ]:
s_tag_counts,s_tag_list = count_tags(df2['squash2_tags'])

In [ ]:
print(len(s_tag_list))
s_tag_counts

In [ ]:
tag_cutoff3 = int(0.05*len(df2.index))

squashed_tags3 = pd.DataFrame(s_tag_counts)
squashed_tags3 = squashed_tags3[(squashed_tags3[0]>tag_cutoff3)]
squash_list3 = list(squashed_tags3.index.values)
print(len(squash_list3))
squashed_tags3

In [ ]:
df3 = df2.copy()
df3['squash3_tags'] = df3['squash2_tags']
df3['squash3_tags'] = df3['squash3_tags'].map(lambda x: squashing(x,squash_list3))
df3 = df3[df3['squash3_tags']!='']

s3_tag_counts,s3_tag_list = count_tags(df3['squash3_tags'])
s3_tag_counts

In [ ]:
# df3[df3['squash3_tags']==df3['squash3_tags'][149]]
# df3[df3['squash3_tags']==float('nan')]
# df3[df3['speaker']=='Norman Foster']

In [ ]:
tag_cutoff4 = 200

squashed_tags4 = pd.DataFrame(s_tag_counts)
squashed_tags4 = squashed_tags4[(squashed_tags4[0]>tag_cutoff4)]
squash_list4 = list(squashed_tags4.index.values)
print(len(squash_list4))
squashed_tags4

In [ ]:
df4 = df3.copy()
df4['squash4_tags'] = df4['squash3_tags']
df4['squash4_tags'] = df4['squash4_tags'].map(lambda x: squashing(x,squash_list4))
df4 = df4[df4['squash4_tags']!='']

s4_tag_counts,s4_tag_list = count_tags(df4['squash4_tags'])
s4_tag_counts

In [ ]:
def cutting(x,cutlist):
#     print(x)
    have = False
    for tag in cutlist:
#         print(tag)
        if tag in x:
            have = True
    if have:
        return x
    else:
        return ''

In [ ]:
df4_cut = df4.copy()
df4_cut['squash4_tags'] = df4_cut['squash4_tags'].map(lambda x: cutting(x,['entertainment','art','future','biodiversity','education']))
df4_cut = df4_cut[df4_cut['squash4_tags']!='']
df4_cut


In [ ]:
s4cut_tag_counts,s4cut_tag_list = count_tags(df4_cut['squash4_tags'])


In [ ]:
s4cut_tag_counts

In [ ]:
tag_cutoff5 = 300

squashed_tags5 = pd.DataFrame(s_tag_counts)
squashed_tags5 = squashed_tags5[(squashed_tags5[0]>tag_cutoff5)]
squash_list5 = list(squashed_tags5.index.values)
print(len(squash_list5))
squashed_tags5

In [ ]:
df5 = df4.copy()
df5['squash5_tags'] = df5['squash4_tags']
df5['squash5_tags'] = df5['squash5_tags'].map(lambda x: squashing(x,squash_list5))
df5 = df5[df5['squash5_tags']!='']

s5_tag_counts,s5_tag_list = count_tags(df5['squash5_tags'])
s5_tag_counts

In [ ]:
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
df3 = df3.reset_index(drop=True)
df4 = df4.reset_index(drop=True)
df5 = df5.reindex(list(range(len(df5))))

In [ ]:
# df1.to_parquet(OUT_DATA_DIR+'cleaned_squashed1.parquet')
# df2.to_parquet(OUT_DATA_DIR+'cleaned_squashed2.parquet')
# df3.to_parquet(OUT_DATA_DIR+'cleaned_squashed3.parquet')
# df4.to_parquet(OUT_DATA_DIR+'cleaned_squashed4.parquet')
# df5.to_parquet(OUT_DATA_DIR+'cleaned_squashed4.parquet')

In [ ]:
df3.iloc[140:150]

In [ ]:
d = pd.DataFrame(['a','b','c'])
d = d.drop(1)

In [ ]:
d = d.reset_index(drop=True)

In [ ]:
d

In [ ]:
df3.iloc[145:150]